In [7]:
!pip install transformers torch numpy -q
#setup
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
from typing import List
import warnings
warnings.filterwarnings('ignore')

print("✓ Packages installed!")

✓ Packages installed!


In [11]:
#Load Model for Poetry

print("Loading GPT-2 Medium for Poetry Generation...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)
model.eval()
tokenizer.pad_token = tokenizer.eos_token

print("✓ Model loaded!")

Loading GPT-2 Medium for Poetry Generation...
Device: cpu
✓ Model loaded!


In [12]:
#POETIC THEMES & VOCABULARIES


print("Defining Poetic Themes...")

# Rich poetic vocabularies for different themes
POETRY_THEMES = {
    'love': [
        'love', 'heart', 'soul', 'passion', 'desire', 'kiss', 'embrace',
        'tender', 'gentle', 'sweet', 'beautiful', 'beloved', 'darling',
        'cherish', 'adore', 'devotion', 'affection', 'romance', 'intimate',
        'longing', 'yearning', 'forever', 'eternal', 'together', 'unite',
        'bond', 'connection', 'warmth', 'caring', 'treasure', 'precious'
    ],

    'nature': [
        'sky', 'earth', 'moon', 'stars', 'sun', 'wind', 'rain', 'clouds',
        'mountains', 'ocean', 'sea', 'waves', 'forest', 'trees', 'flowers',
        'bloom', 'petals', 'garden', 'meadow', 'river', 'stream', 'valley',
        'dawn', 'dusk', 'twilight', 'sunset', 'sunrise', 'nature', 'wild',
        'breeze', 'seasons', 'spring', 'autumn', 'winter', 'summer'
    ],

    'melancholy': [
        'sorrow', 'tears', 'pain', 'sadness', 'lonely', 'alone', 'lost',
        'grief', 'melancholy', 'wistful', 'ache', 'longing', 'memories',
        'fading', 'distant', 'shadows', 'darkness', 'silence', 'empty',
        'broken', 'shattered', 'haunting', 'ghost', 'echo', 'whisper',
        'farewell', 'goodbye', 'absent', 'void', 'hollow', 'cold'
    ],

    'hope': [
        'hope', 'dream', 'light', 'bright', 'shine', 'glowing', 'radiant',
        'tomorrow', 'future', 'promise', 'faith', 'believe', 'strength',
        'courage', 'rise', 'soar', 'freedom', 'wings', 'flight', 'possibilities',
        'infinite', 'boundless', 'new', 'beginning', 'dawn', 'rebirth',
        'inspiration', 'spirit', 'uplift', 'victory', 'triumph', 'joy'
    ],

    'mystical': [
        'magic', 'mystery', 'ethereal', 'celestial', 'cosmic', 'universe',
        'infinite', 'eternal', 'transcendent', 'divine', 'sacred', 'mystical',
        'enchanted', 'spell', 'wonder', 'awe', 'miracle', 'vision', 'dream',
        'spirit', 'phantom', 'shimmer', 'glow', 'mist', 'veil', 'portal',
        'realm', 'dimension', 'destiny', 'fate', 'prophecy', 'ancient'
    ],

    'passion': [
        'fire', 'flame', 'burning', 'desire', 'intense', 'fierce', 'wild',
        'untamed', 'powerful', 'strong', 'bold', 'daring', 'fearless',
        'thunder', 'lightning', 'storm', 'tempest', 'rage', 'fury',
        'ardent', 'fervent', 'zealous', 'vivid', 'brilliant', 'electric',
        'explosive', 'dynamic', 'energy', 'force', 'magnetism', 'allure'
    ]
}

def get_bow_indices(words: List[str], tokenizer) -> List[int]:
    """Enhanced tokenization for poetry words"""
    bow_indices = set()
    for word in words:
        variations = [
            word, word.capitalize(), word.upper(),
            f" {word}", f" {word.capitalize()}",
            f"{word}s", f" {word}s",  # plurals
            f"{word}ing", f" {word}ing",  # gerunds
            f"{word}ed", f" {word}ed"  # past tense
        ]
        for variant in variations:
            tokens = tokenizer.encode(variant, add_special_tokens=False)
            bow_indices.update(tokens)
    return list(bow_indices)

bow_dict = {theme: get_bow_indices(words, tokenizer) for theme, words in POETRY_THEMES.items()}

print(f"✓ Created {len(POETRY_THEMES)} poetic themes")
for theme in POETRY_THEMES:
    print(f"  • {theme}: {len(bow_dict[theme])} tokens")

Defining Poetic Themes...
✓ Created 6 poetic themes
  • love: 311 tokens
  • nature: 337 tokens
  • melancholy: 295 tokens
  • hope: 319 tokens
  • mystical: 334 tokens
  • passion: 293 tokens


In [13]:
# SECTION 4: POETRY-OPTIMIZED PPLM

print("Defining Poetry-Optimized PPLM...")

def compute_bow_loss(logits, bow_indices, device):
    """Compute BoW loss for poetry generation"""
    probs = F.softmax(logits, dim=-1)
    bow_probs = torch.zeros(1).to(device)

    for idx in bow_indices:
        if idx < probs.shape[-1]:
            bow_probs += probs[0, idx]

    loss = -torch.log(bow_probs + 1e-8)
    return loss


def generate_poetry_with_pplm(
    prompt: str,
    theme: str,
    max_length: int = 80,
    num_iterations: int = 15,
    step_size: float = 0.08,
    temperature: float = 0.9,
    top_k: int = 100,
    top_p: float = 0.95,
    repetition_penalty: float = 1.3,
    kl_scale: float = 0.005,
    gm_scale: float = 0.98
):
    """
    Generate poetry with thematic control

    Optimized parameters for creative, poetic generation
    """
    if theme not in bow_dict:
        print(f"⚠ Theme '{theme}' not found. Using 'love'.")
        theme = 'love'

    bow_indices = bow_dict[theme]
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids.clone()

    past_key_values = None
    generated_tokens = set()

    for step in range(max_length):
        # Get original logits
        with torch.no_grad():
            if past_key_values is None:
                outputs = model(generated)
                past_key_values = outputs.past_key_values
                original_logits = outputs.logits[:, -1, :]
            else:
                outputs = model(generated[:, -1:], past_key_values=past_key_values)
                past_key_values = outputs.past_key_values
                original_logits = outputs.logits[:, -1, :]

        original_probs = F.softmax(original_logits, dim=-1)
        perturbed_logits = original_logits.clone()

        # Iterative refinement for thematic control
        for iteration in range(num_iterations):
            current_logits = perturbed_logits.clone().detach().requires_grad_(True)

            # BoW loss
            bow_loss = compute_bow_loss(current_logits, bow_indices, device)

            # KL divergence (very small for more creative freedom)
            current_probs = F.softmax(current_logits, dim=-1)
            kl_loss = F.kl_div(
                F.log_softmax(current_logits, dim=-1),
                original_probs,
                reduction='batchmean'
            )

            total_loss = bow_loss + kl_scale * kl_loss
            total_loss.backward()

            if current_logits.grad is not None:
                grad = current_logits.grad
                grad_norm = torch.norm(grad)

                if grad_norm > 0:
                    normalized_grad = grad / grad_norm
                    perturbed_logits = perturbed_logits - step_size * normalized_grad

        # Geometric fusion
        perturbed_probs = F.softmax(perturbed_logits, dim=-1)
        fused_probs = (perturbed_probs ** gm_scale) * (original_probs ** (1 - gm_scale))
        fused_probs = fused_probs / fused_probs.sum(dim=-1, keepdim=True)
        fused_logits = torch.log(fused_probs + 1e-10)

        # Apply repetition penalty
        for token_id in generated_tokens:
            if token_id < fused_logits.shape[-1]:
                fused_logits[0, token_id] /= repetition_penalty

        # Temperature
        fused_logits = fused_logits / temperature

        # Top-k
        if top_k > 0:
            indices_to_remove = fused_logits < torch.topk(fused_logits, top_k)[0][..., -1, None]
            fused_logits[indices_to_remove] = float('-inf')

        # Top-p (nucleus sampling)
        if top_p < 1.0:
            sorted_logits, sorted_indices = torch.sort(fused_logits, descending=True)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
            fused_logits[indices_to_remove] = float('-inf')

        # Sample
        probs = F.softmax(fused_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        generated_tokens.add(next_token.item())
        generated = torch.cat([generated, next_token], dim=1)

        if next_token.item() == tokenizer.eos_token_id:
            break

    return tokenizer.decode(generated[0], skip_special_tokens=True)


def generate_baseline_poetry(prompt: str, max_length: int = 80):
    """Baseline generation for comparison"""
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + max_length,
            temperature=0.9,
            top_k=100,
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.3,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


def format_as_poem(text: str, line_length: int = 60) -> str:
    """Format generated text as poetry with line breaks"""
    import re

    # Split by punctuation and natural breaks
    sentences = re.split(r'[.!?,;]', text)
    lines = []

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # Split long sentences
        words = sentence.split()
        current_line = []
        current_length = 0

        for word in words:
            if current_length + len(word) + 1 > line_length and current_line:
                lines.append(' '.join(current_line))
                current_line = [word]
                current_length = len(word)
            else:
                current_line.append(word)
                current_length += len(word) + 1

        if current_line:
            lines.append(' '.join(current_line))

    return '\n'.join(lines)

print("✓ Poetry-optimized functions ready!")

Defining Poetry-Optimized PPLM...
✓ Poetry-optimized functions ready!


In [15]:
# SECTION 5: POETRY DEMONSTRATIONS

print("GENERATING POETRY SAMPLES")


# Poetic prompts
poetic_prompts = [
    "In the garden of dreams",
    "Beneath the silver moon",
    "When shadows fall",
    "Through the mist of time"
]

print("\n" + "-"*70)
print("Demonstration: Poetry with Different Themes")
print("-"*70)

for prompt in poetic_prompts[:2]:
    print(f"\n{'='*70}")
    print(f"📝 Prompt: '{prompt}'")
    print('='*70)

    # Show 3 themes
    for theme in ['love', 'nature', 'melancholy']:
        print(f"\n🎭 Theme: [{theme.upper()}]")
        print("-+=+-" * 10)
        poetry = generate_poetry_with_pplm(
            prompt,
            theme=theme,
            max_length=60,
            num_iterations=15,
            step_size=0.08
        )
        formatted = format_as_poem(poetry)
        print(formatted)

GENERATING POETRY SAMPLES

----------------------------------------------------------------------
Demonstration: Poetry with Different Themes
----------------------------------------------------------------------

📝 Prompt: 'In the garden of dreams'

🎭 Theme: [LOVE]
-+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+-
In the garden of dreams We played with baby blue flowers
You know what you do to me now
And now the house's A black meadow And the trees are gray
Beware the caribou Where are they
Beware of the baby

🎭 Theme: [NATURE]
-+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+-
In the garden of dreams I think of the trees in our garden
of dreams — they are in our garden of dreams
" she said
"The trees can breathe
They do things
They look for the light
" So I took her hands
I took them
and they smelled of roses
I didn

🎭 Theme: [MELANCHOLY]
-+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+--+=+-
In the garden of dreams
at midnight we found her
She was carrying her own basket
full of apples

In [17]:
# SECTION 6: THEME ANALYSIS


print("THEMATIC ANALYSIS")


def analyze_theme(text, theme):
    """Analyze theme word usage"""
    text_lower = text.lower()
    theme_words = POETRY_THEMES[theme]
    found = [w for w in theme_words if w in text_lower]
    return found

prompt = "The heart remembers"
print(f"\nPrompt: '{prompt}'\n")

for theme in ['love', 'melancholy', 'hope']:
    poetry = generate_poetry_with_pplm(prompt, theme, max_length=50)
    found = analyze_theme(poetry, theme)
    print(f"\n{theme.upper()}:")
    print(f"  {poetry}")
    print(f"  ✓ Theme words: {', '.join(found[:8])}")

THEMATIC ANALYSIS

Prompt: 'The heart remembers'


LOVE:
  The heart remembers when the heart is dead.

Now there's no heart after all, just dead. You know that.

When you want to make progress in your dead heart's life,

Just keep in mind that it doesn't have the
  ✓ Theme words: heart

MELANCHOLY:
  The heart remembers as though it had been torn, and it beats without a memory, but sometimes a terrible pain and a terrible sight are so close to your consciousness so that you cannot bear them. Many times are you afraid and think you are going crazy or a crazy
  ✓ Theme words: pain

HOPE:
  The heart remembers no names. It is a strange child. Some say it is a boy, some say it is an elf, and some, like the ones at home and at school, say the same thing. Every child wants its name. The heart is not
  ✓ Theme words: 


In [18]:
# SECTION 7: INTERACTIVE POETRY GENERATOR

print("INTERACTIVE POETRY GENERATOR")

def interactive_poetry():
    """Interactive poetry generation"""
    print("\n🎨 Create Your Poem!")
    print(f"Available themes: {', '.join(POETRY_THEMES.keys())}")

    try:
        prompt = input("\n📝 Enter your opening line: ").strip()
        if not prompt:
            prompt = "In the silence of night"
            print(f"Using default: '{prompt}'")

        theme = input(f"🎭 Choose theme ({'/'.join(list(POETRY_THEMES.keys())[:3])}...): ").lower().strip()
        if theme not in POETRY_THEMES:
            theme = 'love'
            print(f"Using '{theme}' theme")

        length = input("📏 Length (short/medium/long) [medium]: ").lower().strip()
        max_len = {'short': 40, 'medium': 60, 'long': 100}.get(length, 60)

        print(f"\n✨ Generating {theme} poetry...")
        print(f"   Prompt: '{prompt}'")
        print(f"   Length: {length}")

        # Generate
        poetry = generate_poetry_with_pplm(
            prompt,
            theme=theme,
            max_length=max_len,
            num_iterations=15,
            step_size=0.08
        )

        # Format and display
        formatted = format_as_poem(poetry)
        print(f"\n{formatted}\n")

        # Analysis
        found = analyze_theme(poetry, theme)
        print(f"📊 Analysis:")
        print(f"   Theme: {theme}")
        print(f"   Words: {len(poetry.split())} words")
        print(f"   Theme words used: {', '.join(found[:10])}")

        # Offer different theme
        print("\n" + "-"*70)
        print("🔄 Same prompt, different theme:")
        alt_theme = input(f"Try another theme ({'/'.join([t for t in POETRY_THEMES.keys() if t != theme][:3])}): ").strip()

        if alt_theme in POETRY_THEMES:
            print(f"\n✨ Generating {alt_theme} version...")
            alt_poetry = generate_poetry_with_pplm(prompt, alt_theme, max_length=max_len)
            alt_formatted = format_as_poem(alt_poetry)
            print(f"\n{alt_formatted}\n")

    except Exception as e:
        print(f"Error: {e}")

# Run interactive demo
interactive_poetry()

INTERACTIVE POETRY GENERATOR

🎨 Create Your Poem!
Available themes: love, nature, melancholy, hope, mystical, passion

✨ Generating hope poetry...
   Prompt: 'the sky is sweer looking today'
   Length: medium

the sky is sweer looking today than the moon and even the
stars have turned to dust
The dust is no longer there
I am now looking north up to the mountains to the South
I now have to walk up to a cloud and the clouds and I don't
even know what clouds are or where I'm going

📊 Analysis:
   Theme: hope
   Words: 59 words
   Theme words used: 

----------------------------------------------------------------------
🔄 Same prompt, different theme:
Try another theme (love/nature/melancholy): melancholy

✨ Generating melancholy version...

the sky is sweer looking today (sorry)
I'll just get out of here and see if I get another chance
" She looked over at him
"Good luck
" He glanced back at the snow and grinned
"See ya at the snow race
I'll be there too
too
" "Thanks
" he replied as he



In [19]:
# Example showcase

print("🌟 EXAMPLE SHOWCASE")


examples = [
    ("Love", "Two hearts"),
    ("Nature", "The mountain stands"),
    ("Mystical", "In the realm of dreams")
]

for theme, prompt in examples:
    print(f"\n{theme.upper()}: '{prompt}'")
    poetry = generate_poetry_with_pplm(prompt, theme.lower(), max_length=45)
    print(format_as_poem(poetry))

print("✅ POETRY GENERATOR READY!")



print("🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭")

🌟 EXAMPLE SHOWCASE

LOVE: 'Two hearts'
Two hearts
One big heart
" they all chanted together
"Do you even remember your family
you
or even the blood or how they died
" the blooded soldiers asked in a bitter tone
They stood
facing

NATURE: 'The mountain stands'
The mountain stands in a valley that's covered in snow
a stark contrast with the lush green of this remote coastal
paradise
In July
the snow melted
allowing the mountain to rise in elevation to the heights
of 1530 m

MYSTICAL: 'In the realm of dreams'
In the realm of dreams and imagination
she and I have been pretty lucky so far
I have loved my first dream
I had a dream I had that I would live among people with
beautiful hair and that would change my life forever
✅ POETRY GENERATOR READY!
🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭🎭
